In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(100,1000), dtype=np.float64) 
sum_resource=np.zeros(shape=(1,1000), dtype=np.float64)
sum_cost=np.zeros(shape=(1,1000), dtype=np.float64)
iteration_update = 0.01
A=np.loadtxt("100_Network_Topology_Double_Random_Adjacency_Matrix_Degree_15.txt") #网络拓扑的邻接矩阵 对角元素要为0
D=np.zeros(shape=(100,100))# 网络的度矩阵
L=np.zeros(shape=(100,100))# 网络的拉普拉斯矩阵 L=D-A
W=np.zeros(shape=(100,100)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
iteration=[]
a=np.loadtxt("100_uniform_distribution_(1,2)_point1.txt")
b=np.loadtxt("100_normal_distribution_(2,0.6)_point1.txt")
agent_gradient=np.zeros(shape=(100,1000), dtype=np.float64) 
bias=np.loadtxt("1000_Gaussian_distribution_(500,10).txt")

In [3]:
#根据邻接矩阵生成网络对应的度矩阵
for i in range(0,100):
    d=0
    for j in range(0,100):
        if A[i][j]==1:
            d=d+1
    D[i][i]=d

#根据邻接矩阵和度矩阵计算拉普拉斯矩阵
L=D-A

#选择用拉普拉斯矩阵作为聚合矩阵
W=L
    
#资源量的初始化 需要给定可行的初始化 即满足平均资源约束
for i in range (0,100):
    agent_resource_quantity[i][0]=50
    
#资源量总和 初始化
sum=0
for i in range (0,100):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

iteration.append(0)

#初始梯度计算
for i in range (0,100):
    if i==7 or i==20 or i==34 or i==47 or i==52 or i==88 or i==66 or i==71 or i==93:
        agent_gradient[i][0]=2*a[i]*agent_resource_quantity[i][0]-2*a[i]*b[i]+bias[0]
    else:
        agent_gradient[i][0]=2*a[i]*agent_resource_quantity[i][0]-2*a[i]*b[i]

#诚实用户成本总和初始化
sum=0
for i in range(0,100):
    sum=sum+a[i]*(agent_resource_quantity[i][0]-b[i])*(agent_resource_quantity[i][0]-b[i])
sum_cost[0][0]=sum

In [4]:
#基于初始化的权重梯度下降无中心资源分配算法
def IWDGD_Gaussian_bias(a,b,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************IWDGD算法+计算误差****************************")
    print("相关变量初始值为：")
    for i in range (0,100):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,1000):#迭代次数
        iteration.append(k)
 
        #所有用户资源申请量基于可行初始化权重梯度下降的迭代  假设部分用户因为设备损坏 导致梯度的计算有误差     
        for i in range (0,100):
            weight_gradient_sum = np.float64(0)
            for j in range (0,100):        
                weight_gradient_sum=weight_gradient_sum+W[i][j]*agent_gradient[j][k-1]
            #weight_gradient_sum = np.round(weight_gradient_sum, decimals=3)
            agent_resource_quantity[i][k]=agent_resource_quantity[i][k-1]-iteration_update*weight_gradient_sum
                #agent_resource_quantity[i][k] = np.round(agent_resource_quantity[i][k], decimals=3)
        
        #计算所有诚实用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,100):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有诚实用户的成本总和
        sum2=np.float64(0)
        for i in range(0,100):
            sum2=sum2+a[i]*(agent_resource_quantity[i][k]-b[i])*(agent_resource_quantity[i][k]-b[i])
        sum_cost[0][k]=sum2
        
        #计算当前迭代的用户梯度
        for i in range (0,100): 
            if i==7 or i==20 or i==34 or i==47 or i==52 or i==88 or i==66 or i==71 or i==93:
                agent_gradient[i][k]=2*a[i]*agent_resource_quantity[i][k]-2*a[i]*b[i]+bias[k]
            else:
                agent_gradient[i][k]=2*a[i]*agent_resource_quantity[i][k]-2*a[i]*b[i]
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%50==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,100):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=1/(pow(k+1,0.5)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
IWDGD_Gaussian_bias(a,b,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias)

****************************IWDGD算法+计算误差****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 50.0
agent 2 的初始资源申请量为: 50.0
agent 3 的初始资源申请量为: 50.0
agent 4 的初始资源申请量为: 50.0
agent 5 的初始资源申请量为: 50.0
agent 6 的初始资源申请量为: 50.0
agent 7 的初始资源申请量为: 50.0
agent 8 的初始资源申请量为: 50.0
agent 9 的初始资源申请量为: 50.0
agent 10 的初始资源申请量为: 50.0
agent 11 的初始资源申请量为: 50.0
agent 12 的初始资源申请量为: 50.0
agent 13 的初始资源申请量为: 50.0
agent 14 的初始资源申请量为: 50.0
agent 15 的初始资源申请量为: 50.0
agent 16 的初始资源申请量为: 50.0
agent 17 的初始资源申请量为: 50.0
agent 18 的初始资源申请量为: 50.0
agent 19 的初始资源申请量为: 50.0
agent 20 的初始资源申请量为: 50.0
agent 21 的初始资源申请量为: 50.0
agent 22 的初始资源申请量为: 50.0
agent 23 的初始资源申请量为: 50.0
agent 24 的初始资源申请量为: 50.0
agent 25 的初始资源申请量为: 50.0
agent 26 的初始资源申请量为: 50.0
agent 27 的初始资源申请量为: 50.0
agent 28 的初始资源申请量为: 50.0
agent 29 的初始资源申请量为: 50.0
agent 30 的初始资源申请量为: 50.0
agent 31 的初始资源申请量为: 50.0
agent 32 的初始资源申请量为: 50.0
agent 33 的初始资源申请量为: 50.0
agent 34 的初始资源申请量为: 50.0
agent 35 的初始资源申请量为: 50.0
agent 36 的初始资源申请量为: 50.0
agent 37 的初始资源申请量为: 50.0
agent

49 次迭代中的结果：
agent 1 在 49 次迭代中的资源申请量为: 51.204710536676615
agent 2 在 49 次迭代中的资源申请量为: 64.57062579583766
agent 3 在 49 次迭代中的资源申请量为: 46.238010262466354
agent 4 在 49 次迭代中的资源申请量为: 68.44286936350966
agent 5 在 49 次迭代中的资源申请量为: 69.24286936350966
agent 6 在 49 次迭代中的资源申请量为: 76.45592272154335
agent 7 在 49 次迭代中的资源申请量为: 41.59951084512985
agent 8 在 49 次迭代中的资源申请量为: 39.724528869268326
agent 9 在 49 次迭代中的资源申请量为: 45.53801026246636
agent 10 在 49 次迭代中的资源申请量为: 59.07447657359244
agent 11 在 49 次迭代中的资源申请量为: 48.088690393031584
agent 12 在 49 次迭代中的资源申请量为: 52.10471053667661
agent 13 在 49 次迭代中的资源申请量为: 69.74286936350966
agent 14 在 49 次迭代中的资源申请量为: 58.47447657359243
agent 15 在 49 次迭代中的资源申请量为: 45.93801026246634
agent 16 在 49 次迭代中的资源申请量为: 46.138010262466345
agent 17 在 49 次迭代中的资源申请量为: 47.48869039303159
agent 18 在 49 次迭代中的资源申请量为: 49.088690393031584
agent 19 在 49 次迭代中的资源申请量为: 43.591112213358066
agent 20 在 49 次迭代中的资源申请量为: 58.97447657359245
agent 21 在 49 次迭代中的资源申请量为: 40.89951084512986
agent 22 在 49 次迭代中的资源申请量为: 41.199510845129865

99 次迭代中的结果：
agent 1 在 99 次迭代中的资源申请量为: 52.21276817224354
agent 2 在 99 次迭代中的资源申请量为: 65.71222038820312
agent 3 在 99 次迭代中的资源申请量为: 47.16320284037999
agent 4 在 99 次迭代中的资源申请量为: 69.62202361610636
agent 5 在 99 次迭代中的资源申请量为: 70.42202361610637
agent 6 在 99 次迭代中的资源申请量为: 77.66172240281192
agent 7 在 99 次迭代中的资源申请量为: 42.45590050758916
agent 8 在 99 次迭代中的资源申请量为: 40.55230142603952
agent 9 在 99 次迭代中的资源申请量为: 46.46320284038
agent 10 在 99 次迭代中的资源申请量为: 60.172816980588934
agent 11 在 99 次迭代中的资源申请量为: 49.053804074975616
agent 12 在 99 次迭代中的资源申请量为: 53.112768172243555
agent 13 在 99 次迭代中的资源申请量为: 70.92202361610637
agent 14 在 99 次迭代中的资源申请量为: 59.57281698058893
agent 15 在 99 次迭代中的资源申请量为: 46.86320284037999
agent 16 在 99 次迭代中的资源申请量为: 47.063202840379994
agent 17 在 99 次迭代中的资源申请量为: 48.453804074975615
agent 18 在 99 次迭代中的资源申请量为: 50.05380407497561
agent 19 在 99 次迭代中的资源申请量为: 44.4799800669966
agent 20 在 99 次迭代中的资源申请量为: 60.07281698058893
agent 21 在 99 次迭代中的资源申请量为: 41.75590050758917
agent 22 在 99 次迭代中的资源申请量为: 42.05590050758916
agent 

149 次迭代中的结果：
agent 1 在 149 次迭代中的资源申请量为: 51.56932037073458
agent 2 在 149 次迭代中的资源申请量为: 64.8915331654256
agent 3 在 149 次迭代中的资源申请量为: 46.62549546197217
agent 4 在 149 次迭代中的资源申请量为: 68.73995430069355
agent 5 在 149 次迭代中的资源申请量为: 69.53995430069357
agent 6 在 149 次迭代中的资源申请量为: 76.71931124530818
agent 7 在 149 次迭代中的资源申请量为: 42.01215067249883
agent 8 在 149 次迭代中的资源申请量为: 40.151339740051505
agent 9 在 149 次迭代中的资源申请量为: 45.925495461972176
agent 10 在 149 次迭代中的资源申请量为: 59.41301300208164
agent 11 在 149 次迭代中的资源申请量为: 48.46468909925667
agent 12 在 149 次迭代中的资源申请量为: 52.46932037073458
agent 13 在 149 次迭代中的资源申请量为: 70.03995430069357
agent 14 在 149 次迭代中的资源申请量为: 58.81301300208163
agent 15 在 149 次迭代中的资源申请量为: 46.32549546197217
agent 16 在 149 次迭代中的资源申请量为: 46.52549546197217
agent 17 在 149 次迭代中的资源申请量为: 47.86468909925667
agent 18 在 149 次迭代中的资源申请量为: 49.46468909925667
agent 19 在 149 次迭代中的资源申请量为: 43.990703063787876
agent 20 在 149 次迭代中的资源申请量为: 59.31301300208165
agent 21 在 149 次迭代中的资源申请量为: 41.31215067249884
agent 22 在 149 次迭代中的资源申请量为: 

199 次迭代中的结果：
agent 1 在 199 次迭代中的资源申请量为: 50.926644063675454
agent 2 在 199 次迭代中的资源申请量为: 64.18401412945403
agent 3 在 199 次迭代中的资源申请量为: 46.02998058741545
agent 4 在 199 次迭代中的资源申请量为: 68.01694639855906
agent 5 在 199 次迭代中的资源申请量为: 68.81694639855905
agent 6 在 199 次迭代中的资源申请量为: 75.98756245540443
agent 7 在 199 次迭代中的资源申请量为: 41.462269344152105
agent 8 在 199 次迭代中的资源申请量为: 39.623257286409164
agent 9 在 199 次迭代中的资源申请量为: 45.32998058741545
agent 10 在 199 次迭代中的资源申请量为: 58.72512315439229
agent 11 在 199 次迭代中的资源申请量为: 47.84563819972906
agent 12 在 199 次迭代中的资源申请量为: 51.82664406367545
agent 13 在 199 次迭代中的资源申请量为: 69.31694639855904
agent 14 在 199 次迭代中的资源申请量为: 58.125123154392284
agent 15 在 199 次迭代中的资源申请量为: 45.72998058741544
agent 16 在 199 次迭代中的资源申请量为: 45.92998058741544
agent 17 在 199 次迭代中的资源申请量为: 47.24563819972907
agent 18 在 199 次迭代中的资源申请量为: 48.84563819972907
agent 19 在 199 次迭代中的资源申请量为: 43.41831032673865
agent 20 在 199 次迭代中的资源申请量为: 58.6251231543923
agent 21 在 199 次迭代中的资源申请量为: 40.7622693441521
agent 22 在 199 次迭代中的资源申请量为: 

...

1709 次迭代中的结果：
agent 1 在 1709 次迭代中的资源申请量为: 51.1560500919772
agent 2 在 1709 次迭代中的资源申请量为: 64.42294603711409
agent 3 在 1709 次迭代中的资源申请量为: 46.25176217386659
agent 4 在 1709 次迭代中的资源申请量为: 68.25839868403203
agent 5 在 1709 次迭代中的资源申请量为: 69.05839868403203
agent 6 在 1709 次迭代中的资源申请量为: 76.23057115858289
agent 7 在 1709 次迭代中的资源申请量为: 41.67482340937927
agent 8 在 1709 次迭代中的资源申请量为: 39.83045470358878
agent 9 在 1709 次迭代中的资源申请量为: 45.55176217386659
agent 10 在 1709 次迭代中的资源申请量为: 58.96110345802523
agent 11 在 1709 次迭代中的资源申请量为: 48.07139768749393
agent 12 在 1709 次迭代中的资源申请量为: 52.05605009197722
agent 13 在 1709 次迭代中的资源申请量为: 69.55839868403204
agent 14 在 1709 次迭代中的资源申请量为: 58.361103458025234
agent 15 在 1709 次迭代中的资源申请量为: 45.9517621738666
agent 16 在 1709 次迭代中的资源申请量为: 46.1517621738666
agent 17 在 1709 次迭代中的资源申请量为: 47.47139768749394
agent 18 在 1709 次迭代中的资源申请量为: 49.07139768749394
agent 19 在 1709 次迭代中的资源申请量为: 43.63571006702802
agent 20 在 1709 次迭代中的资源申请量为: 58.861103458025234
agent 21 在 1709 次迭代中的资源申请量为: 40.97482340937928
agent 22 

1769 次迭代中的结果：
agent 1 在 1769 次迭代中的资源申请量为: 51.580309585099386
agent 2 在 1769 次迭代中的资源申请量为: 65.03968330545185
agent 3 在 1769 次迭代中的资源申请量为: 46.570537930214634
agent 4 在 1769 次迭代中的资源申请量为: 68.95022860935958
agent 5 在 1769 次迭代中的资源申请量为: 69.75022860935958
agent 6 在 1769 次迭代中的资源申请量为: 77.00426469386643
agent 7 在 1769 次迭代中的资源申请量为: 41.903600344703946
agent 8 在 1769 次迭代中的资源申请量为: 40.0194768425187
agent 9 在 1769 次迭代中的资源申请量为: 45.870537930214624
agent 10 在 1769 次迭代中的资源申请量为: 59.508601179855006
agent 11 在 1769 次迭代中的资源申请量为: 48.440852368886084
agent 12 在 1769 次迭代中的资源申请量为: 52.48030958509939
agent 13 在 1769 次迭代中的资源申请量为: 70.25022860935958
agent 14 在 1769 次迭代中的资源申请量为: 58.90860117985501
agent 15 在 1769 次迭代中的资源申请量为: 46.270537930214616
agent 16 在 1769 次迭代中的资源申请量为: 46.47053793021462
agent 17 在 1769 次迭代中的资源申请量为: 47.84085236888608
agent 18 在 1769 次迭代中的资源申请量为: 49.44085236888608
agent 19 在 1769 次迭代中的资源申请量为: 43.907670630875344
agent 20 在 1769 次迭代中的资源申请量为: 59.40860117985502
agent 21 在 1769 次迭代中的资源申请量为: 41.20360034470394
a

1829 次迭代中的结果：
agent 1 在 1829 次迭代中的资源申请量为: 51.59621220649803
agent 2 在 1829 次迭代中的资源申请量为: 64.99823926338473
agent 3 在 1829 次迭代中的资源申请量为: 46.61985127586765
agent 4 在 1829 次迭代中的资源申请量为: 68.88355301548516
agent 5 在 1829 次迭代中的资源申请量为: 69.68355301548517
agent 6 在 1829 次迭代中的资源申请量为: 76.9061446038833
agent 7 在 1829 次迭代中的资源申请量为: 41.98617568595017
agent 8 在 1829 次迭代中的资源申请量为: 40.11850480398509
agent 9 在 1829 次迭代中的资源申请量为: 45.91985127586765
agent 10 在 1829 次迭代中的资源申请量为: 59.48835336963141
agent 11 在 1829 次迭代中的资源申请量为: 48.47354918881423
agent 12 在 1829 次迭代中的资源申请量为: 52.49621220649803
agent 13 在 1829 次迭代中的资源申请量为: 70.18355301548517
agent 14 在 1829 次迭代中的资源申请量为: 58.88835336963141
agent 15 在 1829 次迭代中的资源申请量为: 46.31985127586764
agent 16 在 1829 次迭代中的资源申请量为: 46.519851275867644
agent 17 在 1829 次迭代中的资源申请量为: 47.87354918881423
agent 18 在 1829 次迭代中的资源申请量为: 49.47354918881422
agent 19 在 1829 次迭代中的资源申请量为: 43.973613630707156
agent 20 在 1829 次迭代中的资源申请量为: 59.38835336963142
agent 21 在 1829 次迭代中的资源申请量为: 41.28617568595016
agent 2

1889 次迭代中的结果：
agent 1 在 1889 次迭代中的资源申请量为: 52.012669913510486
agent 2 在 1889 次迭代中的资源申请量为: 65.40995435872301
agent 3 在 1889 次迭代中的资源申请量为: 47.01047001262018
agent 4 在 1889 次迭代中的资源申请量为: 69.27805170744328
agent 5 在 1889 次迭代中的资源申请量为: 70.0780517074433
agent 6 在 1889 次迭代中的资源申请量为: 77.27420944650345
agent 7 在 1889 次迭代中的资源申请量为: 42.336935611457534
agent 8 在 1889 次迭代中的资源申请量为: 40.44619756191893
agent 9 在 1889 次迭代中的资源申请量为: 46.310470012620186
agent 10 在 1889 次迭代中的资源申请量为: 59.90882873288217
agent 11 在 1889 次迭代中的资源申请量为: 48.87930389491128
agent 12 在 1889 次迭代中的资源申请量为: 52.912669913510506
agent 13 在 1889 次迭代中的资源申请量为: 70.5780517074433
agent 14 在 1889 次迭代中的资源申请量为: 59.30882873288217
agent 15 在 1889 次迭代中的资源申请量为: 46.710470012620185
agent 16 在 1889 次迭代中的资源申请量为: 46.91047001262019
agent 17 在 1889 次迭代中的资源申请量为: 48.279303894911294
agent 18 在 1889 次迭代中的资源申请量为: 49.879303894911295
agent 19 在 1889 次迭代中的资源申请量为: 44.34561138393394
agent 20 在 1889 次迭代中的资源申请量为: 59.80882873288218
agent 21 在 1889 次迭代中的资源申请量为: 41.63693561145754
age

1959 次迭代中的结果：
agent 1 在 1959 次迭代中的资源申请量为: 51.55548349567785
agent 2 在 1959 次迭代中的资源申请量为: 64.79235199605725
agent 3 在 1959 次迭代中的资源申请量为: 46.66076218464539
agent 4 在 1959 次迭代中的资源申请量为: 68.60972834708008
agent 5 在 1959 次迭代中的资源申请量为: 69.40972834708009
agent 6 在 1959 次迭代中的资源申请量为: 76.55710287487574
agent 7 在 1959 次迭代中的资源申请量为: 42.089984739592204
agent 8 在 1959 次迭代中的资源申请量为: 40.248085598589554
agent 9 在 1959 次迭代中的资源申请量为: 45.96076218464538
agent 10 在 1959 次迭代中的资源申请量为: 59.34370224600217
agent 11 在 1959 次迭代中的资源申请量为: 48.476224856754186
agent 12 在 1959 次迭代中的资源申请量为: 52.455483495677846
agent 13 在 1959 次迭代中的资源申请量为: 69.90972834708008
agent 14 在 1959 次迭代中的资源申请量为: 58.743702246002165
agent 15 在 1959 次迭代中的资源申请量为: 46.36076218464537
agent 16 在 1959 次迭代中的资源申请量为: 46.56076218464537
agent 17 在 1959 次迭代中的资源申请量为: 47.876224856754185
agent 18 在 1959 次迭代中的资源申请量为: 49.47622485675418
agent 19 在 1959 次迭代中的资源申请量为: 44.04806004822386
agent 20 在 1959 次迭代中的资源申请量为: 59.24370224600217
agent 21 在 1959 次迭代中的资源申请量为: 41.38998473959219
ag

In [6]:
np.savetxt('../data/03-9-bias-agent-agent_resource_quantity_large_value_Gaussian_bias(500,10).txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/03-9-bias-agent-sum_cost_large_value_Gaussian_bias(500,10).txt',sum_cost,fmt='%0.8f')#用户成本总和 
np.savetxt('../data/03-9-bias-agent-sum_resource_large_value_Gaussian_bias(500,10).txt',sum_resource,fmt='%0.8f')#用户资源总和 
np.savetxt('../data/03-9-bias-agent-agent_gradient_large_value_Gaussian_bias(500,10).txt',agent_gradient,fmt='%0.8f')#用户迭代梯度 